In [2]:
import matplotlib.pyplot as plt
from ultralytics import YOLO
from PIL import Image
from generate_perturbation import generate_perturbation
from transforms import resize_and_pad, image_to_tensor
from tog_modules import TOG_DetectionValidator, TOG_DetectionPredictor

In [3]:
# Load a model
model_1 = YOLO("yolo11n.pt")
model_2 = YOLO("yolo11n.pt")
data = [
    "./datasets/coco8/images/val/000000000036.jpg",
    "./datasets/coco8/images/val/000000000042.jpg",
    "./datasets/coco8/images/val/000000000049.jpg",
    "./datasets/coco8/images/val/000000000061.jpg"
]

In [4]:
model = YOLO("yolo11n.pt")
metrics = model.val(data='coco8.yaml', imgsz=640, device=0, validator=TOG_DetectionValidator)
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP75: {metrics.box.map75:.4f}")

Ultralytics 8.3.28 🚀 Python-3.12.7 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients

Dataset 'coco8.yaml' images not found ⚠️, missing path '/home/liuhailin/codes/math_ML/datasets/coco8/images/val'


100%|██████████| 433k/433k [00:00<00:00, 3.04MB/s]
Unzipping /home/liuhailin/codes/math_ML/datasets/coco8.zip to /home/liuhailin/codes/math_ML/datasets/coco8...: 100%|██████████| 25/25 [00:00<00:00, 6658.47file/s]

Dataset download success ✅ (1.5s), saved to /home/liuhailin/codes/math_ML/datasets




val: Scanning /home/liuhailin/codes/math_ML/datasets/coco8/labels/val... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<00:00, 484.60it/s]

val: New cache created: /home/liuhailin/codes/math_ML/datasets/coco8/labels/val.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.40s/it]


                   all          4         17      0.167     0.0167     0.0917     0.0275
                person          3         10          1        0.1       0.55      0.165
                   dog          1          1          0          0          0          0
                 horse          1          2          0          0          0          0
              elephant          1          2          0          0          0          0
              umbrella          1          1          0          0          0          0
          potted plant          1          1          0          0          0          0
Speed: 768.1ms preprocess, 16.4ms inference, 0.0ms loss, 30.2ms postprocess per image
Results saved to runs/detect/val19
mAP50-95: 0.0275
mAP50: 0.0917
mAP75: 0.0000


In [5]:
import numpy as np
for i in range(1, 26):
    image_input = Image.open("datasets/test/" + str(i) + ".png")
    origin = image_to_tensor(resize_and_pad(image_input, 640))
    attacked, noise = generate_perturbation(origin)
    img_att = Image.fromarray(np.uint8(attacked.cpu().numpy().transpose((0, 2, 3, 1))[0]*256))
    img_att.save("datasets/adv_test/" + str(i) + ".png")

In [6]:
import numpy as np
fig, axs = plt.subplots(figsize=(15, 15), nrows=5, ncols=len(data))
for i in range(len(data)):
    image_input = Image.open(data[i])
    origin = image_to_tensor(resize_and_pad(image_input, 640))
    attacked, noise = generate_perturbation(origin)
    img_att = Image.fromarray(np.uint8(attacked.cpu().numpy().transpose((0, 2, 3, 1))[0]*256))
    img_att.save("test.jpg")
    
    result_origin = model_1.predict(origin, imgsz=(640, 640))
    result_attacked = model_2.predict(origin, imgsz=(640, 640), predictor=TOG_DetectionPredictor)
    axs[0, i].imshow(origin.numpy().transpose((0, 2, 3, 1))[0])
    axs[0, i].set_title("Origin Image")
    axs[0, i].axis('off')
    axs[1, i].imshow(attacked.cpu().numpy().transpose((0, 2, 3, 1))[0])
    axs[1, i].set_title("Attacked Detection")
    axs[1, i].axis('off')
    noise_zoomed = noise.cpu().numpy().transpose((0, 2, 3, 1))[0]
    axs[2, i].imshow((noise_zoomed - noise_zoomed.min()) / (noise_zoomed.max() - noise_zoomed.min()))
    axs[2, i].set_title("Attack Noise")
    axs[2, i].axis('off')
    axs[3, i].imshow(result_origin[0].plot(line_width=2))
    axs[3, i].set_title("Normal Result")
    axs[3, i].axis('off')
    axs[4, i].imshow(result_attacked[0].plot(line_width=2))
    axs[4, i].set_title("Attacked Result")
    axs[4, i].axis('off')



0: 640x640 1 person, 1 umbrella, 13.8ms
Speed: 0.8ms preprocess, 13.8ms inference, 6.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.9ms
Speed: 714.0ms preprocess, 9.9ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 dog, 1 suitcase, 11.5ms
Speed: 1.1ms preprocess, 11.5ms inference, 8.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.8ms
Speed: 744.9ms preprocess, 9.8ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 4 persons, 2 horses, 9.8ms
Speed: 0.7ms preprocess, 9.8ms inference, 4.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.2ms
Speed: 748.8ms preprocess, 10.2ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 dogs, 10.4ms
Speed: 0.8ms preprocess, 10.4ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.4ms
Speed: 701.0ms preprocess, 9.4ms

In [7]:
plt.savefig("./output/output.png")
plt.show()

<Figure size 1500x1500 with 20 Axes>

In [9]:
model = YOLO("yolo11n.pt")
metrics = model.val(data='coco.yaml', imgsz=640, device=0, validator=TOG_DetectionValidator)
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP75: {metrics.box.map75:.4f}")

Ultralytics 8.3.28 🚀 Python-3.12.7 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)


YOLO11n summary (fused): 238 layers, 2,616,248 parameters, 0 gradients


val: Scanning /home/liuhailin/codes/math_ML/datasets/coco/labels/val2017... 4952 images, 48 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:05<00:00, 834.02it/s]


val: New cache created: /home/liuhailin/codes/math_ML/datasets/coco/labels/val2017.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [1:08:43<00:00, 13.17s/it]


                   all       5000      36335      0.029     0.0522     0.0244     0.0138
                person       2693      10777     0.0852      0.165     0.0807     0.0378
               bicycle        149        314     0.0183     0.0573     0.0158    0.00812
                   car        535       1918     0.0548     0.0386     0.0354     0.0193
            motorcycle        159        367     0.0557     0.0708      0.063     0.0279
              airplane         97        143     0.0453     0.0839     0.0484     0.0239
                   bus        189        283      0.108     0.0601     0.0873     0.0486
                 train        157        190     0.0329      0.132     0.0474     0.0206
                 truck        250        414     0.0394      0.099     0.0316     0.0146
                  boat        121        424    0.00924     0.0118    0.00632    0.00292
         traffic light        191        634     0.0169     0.0158     0.0138       0.01
          fire hydran